#### Preparing data for modelling clustering on Population, Number of Hospitals and Number of Companies. 

In [1]:
import numpy as np
import pandas as pd

In [19]:
company = pd.ExcelFile("D:/UTD_JSOM/Spring 2018/Practicum/Market Analysis/donations/donations_company_2015.xlsx")

In [20]:
company.sheet_names

['donations', 'company']

In [21]:
comp = pd.read_excel("D:/UTD_JSOM/Spring 2018/Practicum/Market Analysis/donations/donations_company_2015.xlsx",
                    sheetname='company')

In [23]:
comp.columns

Index(['Uni_code', 'FIPS STATE   CODE', 'STATE DESCRIPTION',
       'FIPS COUNTY   CODE ', 'COUNTY DESCRIPTION',
       'ENTERPRISE EMPLOYMENT SIZE', 'NUMBER OF FIRMS',
       'NUMBER OF ESTABLISHMENTS', 'EMPLOYMENT', 'EMPLOYMENT RANGE FLAG',
       'EMPLOYMENT NOISE FLAG', 'ANNUAL PAYROLL ($1,000)',
       'ANNUAL PAYROLL NOISE FLAG'],
      dtype='object')

In [25]:
comp['ENTERPRISE EMPLOYMENT SIZE'].value_counts()

01:  Total      3192
02:  <20        3179
03:  20-99      3172
05:  500+       3166
04:  100-499    3145
Name: ENTERPRISE EMPLOYMENT SIZE, dtype: int64

In [27]:
comp.rename(columns={'STATE DESCRIPTION':'State','COUNTY DESCRIPTION':'County',
                    'ENTERPRISE EMPLOYMENT SIZE':'Employment_Size','NUMBER OF FIRMS':'Number_Firms'},
           inplace=True)

In [28]:
comp.columns

Index(['Uni_code', 'FIPS STATE   CODE', 'State', 'FIPS COUNTY   CODE ',
       'County', 'Employment_Size', 'Number_Firms', 'NUMBER OF ESTABLISHMENTS',
       'EMPLOYMENT', 'EMPLOYMENT RANGE FLAG', 'EMPLOYMENT NOISE FLAG',
       'ANNUAL PAYROLL ($1,000)', 'ANNUAL PAYROLL NOISE FLAG'],
      dtype='object')

In [26]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [34]:
q='''
Select Uni_code, State, County, 
sum(CASE when Employment_Size = '02:  <20' then Number_Firms else 0 end ) as Firms_20,
sum(CASE when Employment_Size = '03:  20-99' then Number_Firms else 0 end ) as Firms_20_99,
sum(CASE when Employment_Size = '04:  100-499' then Number_Firms else 0 end ) as Firms_100_499,
sum(CASE when Employment_Size = '05:  500+' then Number_Firms else 0 end ) as Firms_500
from comp
group by Uni_code;
'''

company_data = pysqldf(q)

In [35]:
company_data.head()

,Uni_code,State,County,Firms_20,Firms_20_99,Firms_100_499,Firms_500
0,11,Alabama,Autauga,526,75,35,152
1,13,Alabama,Baldwin,3476,432,127,396
2,15,Alabama,Barbour,295,20,21,72
3,17,Alabama,Bibb,184,31,17,31
4,19,Alabama,Blount,487,48,20,62


In [36]:
donate = pd.read_excel("D:/UTD_JSOM/Spring 2018/Practicum/Market Analysis/donations/donations_company_2015.xlsx",
                    sheetname='donations')

In [38]:
donate = donate[np.isfinite(donate['cbsa_code'])]

In [41]:
q = '''
Select d.*,c.* from donate as d 
inner join company_data as c 
on d.Uniq_state_countyfips = c.Uni_code;
'''

donate_company = pysqldf(q)

In [198]:
# Considering the areas in cbsa where the population is max.
pop = donate_company.iloc[:,[2,5,6]]

q = '''
select cbsa_code, Uniq_state_countyfips, max(cy_pop_2015) as population
from pop
group by cbsa_code;
'''

pop_max = pysqldf(q)

### Combining the donations data with the total number of company on fips.

In [190]:
donate_company_final = pop_max.set_index('Uniq_state_countyfips').join(donate_company.set_index('Uniq_state_countyfips'),how='inner',lsuffix = "_")

### Combining the merged data of donations and number of company with the number of hospitals based on cbsa code 

In [201]:
hospitals = pd.read_csv("D:/UTD_JSOM/Spring 2018/Practicum/Market Analysis/hospital_count.csv")

In [207]:
donate_company_final = donate_company_final.set_index(['cbsa_code'])

In [208]:
donate_company_hospitals = donate_company_final.join(hospitals.set_index('CBSA_CODE'),
                                                     how='inner',lsuffix = "_")

In [213]:
donate_company_hospitals.rename(columns={'Provider ID': 'Number_Hospitals'},inplace=True)

In [214]:
donate_company_hospitals.drop(['cbsa_code_'],axis=1,inplace=True)

In [216]:
donate_company_hospitals = donate_company_hospitals.reset_index()

In [217]:
### Exporting the file to local directory

donate_company_hospitals.to_csv('D:/UTD_JSOM/Spring 2018/Practicum/Market Analysis/donation_company_hos2015.csv')